In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# from utils import 

In [2]:
# filter out movies with tags
feat_csv = pd.read_csv('mv/features.csv')

# movie_table = pd.read_csv('mv/movies.csv')
# movie_table = movie_table[movie_table.MovieID.isin(feat_csv.mvid)]
# movie_table.to_csv('mv/movie_with_tags.csv')

In [2]:
movie = pd.read_csv('mv/movie_with_tags.csv')
rating = pd.read_csv('mv/ratings.dat', sep='::', engine='python', header=None,
                          names=["UserID", "MovieID", "Rating", "Timestamp"])
threshold = 10
year = 1997
time_stamp = datetime(year=year, month=1, day=1).timestamp()
movie = movie[movie.Year < year]
rating = rating[rating.Timestamp < time_stamp]

AdjRaw = np.zeros((movie.shape[0], movie.shape[0]), dtype='uint16')
idSet = {key: i for i, key in enumerate(movie.MovieID)}
user_set = rating.groupby("UserID")["MovieID"].apply(list).reset_index(name='mvList')

for _, row in user_set.iterrows():
    for mv in row.mvList:
        try:
            x = idSet[mv]
            for peer in row.mvList:
                if peer == mv:
                    pass
                elif peer in idSet:
                    y = idSet[peer]
                    AdjRaw[x, y] += 1
                    AdjRaw[y, x] += 1
        except:
            pass
Adj_mat = (AdjRaw > threshold).astype(int)
print("Build complete.")

with open('mv/adj_list_y1996.txt', 'w') as f:
    for k, v in idSet.items():
        for mv in movie.MovieID[Adj_mat[v] == 1]:
            f.write("{} {}\n".format(k, mv))
print("Write to disk complete.")

Build complete.
Write to disk complete.


In [28]:
# directly use adj list

# for _, row in user_set.iterrows():
#     for me in row.mvList:
#         for peer in row.mvList:
#             if peer != me:
#                 try:
#                     movie_list[me] |= {peer}
#                 except: 
#                     pass

In [3]:
# obj_name = "mv/rating-y1995-t10.obj"
# mvg = load_graph(obj_name)
feat_csv = pd.read_csv('mv/features.csv')
feat_csv = feat_csv[feat_csv.mvid.isin(idSet.keys())]

In [4]:
# with open('mv/features_y1996.txt', 'w') as f:
def to_int(line):
    line = line[1:-1].split(", ")
    line = [int(float(i)) for i in line]
    return line

In [5]:
feat_csv.features = feat_csv.features.apply(to_int)

In [6]:
for _, row in feat_csv.iterrows():
    print(len(row.features))
    break


16007


In [ ]:
num_nodes = 4412  # Adj_mat.shape[0]
num_feats = 16007

In [7]:
with open('mv/features_y1996.txt', 'w') as f:
    for _, row in feat_csv.iterrows():
        f.write(str(row.mvid))
        for x in row.features:
            f.write(" {}".format(x))
        f.write('\n')

In [8]:
def to_int2(line):
    line = line[1:-1].split(", ")
    line = [int(i) for i in line]
    return line

def normalize(line):
    line = np.array(line)
    return line / sum(line)

In [9]:
movie.gencode = movie.gencode.apply(to_int2)
# movie.gencode = movie.gencode.apply(normalize)

In [10]:
with open('mv/labels_y1996.txt', 'w') as f:
    for _, row in movie.iterrows():
        f.write(str(row.MovieID))
        for x in row.gencode:
            f.write(" {}".format(x))
        f.write('\n')

In [18]:
a = [1, 2, 3,4, 5]
a[1:]
list(map(float, a[1:]))

[2.0, 3.0, 4.0, 5.0]

In [11]:
!awk 'NF' mv/features_y1996.txt > mv/y1996.feature
!awk 'NF' mv/labels_y1996.txt > mv/y1996.label
!awk 'NF' mv/adj_list_y1996.txt > mv/y1996.adj.list

In [12]:
from collections import defaultdict
def normalize(line):
    line = np.array(line)
    if sum(line) == 0:
        return line
    else:
        return line / sum(line)

def load_mv():
    
    num_nodes = 4412  # wc -l cora/cora.content 2708
    num_feats = 16007
    feat_data = np.zeros((num_nodes, num_feats))
    labels = np.empty((num_nodes, 19))  # num_nodes x 19
    # maps are used to index
    node_map = {}
    with open("mv/y1996.feature") as fp:
        for i, line in enumerate(fp):
            info = line.strip().split()  # len(info) = 1435
            feat_data[i, :] = normalize(list(map(float, info[1:])))  # not include head
            node_map[info[0]] = i
            
    with open("mv/y1996.label") as fp:
        for i, line in enumerate(fp):
            info = line.strip().split()
            labels[i, :] = list(map(float, info[1:]))

    adj_lists = defaultdict(set)  # so that each value in adj_lists is a set
    with open("mv/y1996.adj.list") as fp:
        for i, line in enumerate(fp):
            info = line.strip().split()
            mv1 = node_map[info[0]]
            mv2 = node_map[info[1]]
            adj_lists[mv1].add(mv2)
            adj_lists[mv2].add(mv1)
    return feat_data, labels, adj_lists

In [ ]:
a, b, c = load_mv()

In [30]:
c

defaultdict(set, {})

## Evaluate result

In [2]:
import numpy as np

In [30]:
lb_val = np.load('labels_val.npz.npy')
val_out = np.load('val_output.npz.npy')

In [36]:
val_out2 = np.exp(val_out) - 1

In [37]:
lb_val[lb_val>0] = 1
val_out2[val_out2>0] = 1
val_out2[val_out2<0] = 0
# hamming_loss(lb_val, val_out)

In [9]:
from sklearn.metrics import hamming_loss

In [26]:
lb_val[lb_val>0] = 1
val_out[val_out>0] = 1
val_out[val_out<0] = 0
# hamming_loss(lb_val, val_out)

In [27]:
val_out[0]

array([1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0.], dtype=float32)

In [29]:
hamming_loss(lb_val, val_out)

0.1876842105263158

In [38]:
hamming_loss(lb_val, val_out2)

0.1876842105263158